In [ ]:
# In test.ipynb

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import your visualization functions
from src.visualisation import plot_weight_heatmap, plot_bar_chart_for_factor, plot_comparison, plot_difference_and_ratio

# Load the saved linear weights
W_loaded = np.load('saved_models/linear_weights.npy')
print("Loaded weight matrix shape:", W_loaded.shape)

# Define your feature names and factor labels
feature_names = [
    'm/Q 28', 'm/Q 29', 'm/Q 30', 'm/Q 31', 'm/Q 37', 'm/Q 38', 'm/Q 41',
    'm/Q 42', 'm/Q 43', 'm/Q 44', 'm/Q 45', 'm/Q 48', 'm/Q 49', 'm/Q 50',
    'm/Q 51', 'm/Q 52', 'm/Q 53', 'm/Q 54', 'm/Q 55', 'm/Q 56', 'm/Q 57',
    'm/Q 58', 'm/Q 59', 'm/Q 60', 'm/Q 61', 'm/Q 62', 'm/Q 63', 'm/Q 64',
    'm/Q 65', 'm/Q 66', 'm/Q 67', 'm/Q 68', 'm/Q 69', 'm/Q 70', 'm/Q 71',
    'm/Q 72', 'm/Q 73', 'm/Q 74', 'm/Q 75', 'm/Q 76', 'm/Q 77', 'm/Q 78',
    'm/Q 79'
]
# Construct labels based on the number of factors (rows) in W_loaded
factor_labels = [f'Factor {i+1}' for i in range(W_loaded.shape[0])]

# Use one of the visualization functions to plot a heatmap of the weights
plot_weight_heatmap(W_loaded, feature_names, factor_labels)

# Similarly, you can invoke other functions like plot_bar_chart_for_factor or plot_difference_and_ratio,
# depending on the data you want to compare. For instance, to plot a bar chart for each factor:
for i in range(W_loaded.shape[0]):
    plot_bar_chart_for_factor(W_loaded[i, :], feature_names, f"Factor {i+1} Contribution Profile")


In [ ]:
# Comparative Visualization Code Block in test.ipynb

# -------------------------
# 1. Load NMF Factor Profiles
# -------------------------
nmf_df = pd.read_csv("NMF_Factor_Profiles.csv", index_col=0)
print("Shape of NMF factor profiles:", nmf_df.shape)
print("Preview of NMF factor profiles:")
display(nmf_df.head())

# If the row index is just numeric (0, 1, 2), rename them for clarity:
nmf_df.index = ["NMF_Factor1", "NMF_Factor2", "NMF_Factor3"]

# -------------------------
# 2. Load Autoencoder Weights (AE Factors)
# -------------------------
# Load the previously saved linear weights from your training
W_auto = W_loaded
# Create a DataFrame for AE factors using the same columns as the NMF CSV
AE_df = pd.DataFrame(W_auto,
                     index=["AE_Factor1", "AE_Factor2", "AE_Factor3"],
                     columns=nmf_df.columns)

# -------------------------
# 3. Normalize Both DataFrames (Row Normalization)
# -------------------------
def row_normalize_sum(df):
    """
    Row-normalize the DataFrame so that each row sums to 1.
    Rows that sum to 0 remain unchanged.
    """
    df_norm = df.copy()
    for idx in df_norm.index:
        row_sum = df_norm.loc[idx].sum()
        if row_sum != 0:
            df_norm.loc[idx] = df_norm.loc[idx] / row_sum
    return df_norm

nmf_df_norm = row_normalize_sum(nmf_df)
ae_df_norm = row_normalize_sum(AE_df)

print("Row sums for NMF factors:")
print(nmf_df_norm.sum(axis=1))
print("Row sums for AE factors:")
print(ae_df_norm.sum(axis=1))

# -------------------------
# 4. Plot Comparative Visualizations
# -------------------------
# Plot the difference and ratio between a selected pair of factors
plot_difference_and_ratio(nmf_df_norm, ae_df_norm,
                          nmf_factor="NMF_Factor1",
                          ae_factor="AE_Factor1")

# Alternatively, display a full comparison heatmap and scatter grid
plot_comparison(nmf_df_norm, ae_df_norm, title_suffix=": NMF vs AE Factor Profiles")
